In [3]:
import pandas as pd
import numpy as np
import csv

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

from datetime import datetime

d = datetime.today().strftime('%Y%m%d-%H%M%S')

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

In [6]:
import glob

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/acs_results/NC/'
files = glob.glob(f'{data_dir}/*.csv')

first_df = None
for i,f in enumerate(files):
    this_df = pd.read_csv(f)
    if first_df is None:
        first_df = this_df
    else:
        df = pd.merge(first_df, this_df, on='GEOID', suffixes=(f'_x{i}', f'_x{i+1}'))
        first_df = df
        
        
        
        
# first_df = None
# for each in dfs:
#     if first_df is None:
#         first_df = each
#     else:
#         df = pd.merge(first_df, each, on='GEOID')
#         first_df = df
# df

In [7]:
df

,GEOID,POP_x2,POP_M,POP_F,POP_A0004,POP_A0509,POP_A1014,POP_A1517,POP_A1819,POP_A20,...,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,37001,169185,80687,88498,9733,10441,10964,6617,6695,2813,...,72651,48324,2545,1702,2606,3574,2523,2113,836,8428
1,37003,36491,18568,17923,1600,1691,2675,1424,696,377,...,15997,11094,115,243,162,69,110,12,0,4192
2,37005,10910,5372,5538,454,595,473,342,197,88,...,7721,6094,44,100,18,162,74,14,0,1215
3,37007,22388,11299,11089,1122,1153,1536,810,538,225,...,10027,7110,74,108,117,242,42,78,34,2222
4,37009,26598,13095,13503,1066,1182,1508,869,521,281,...,17006,12448,111,360,398,252,204,101,65,3067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,37191,117692,57965,59727,7572,7298,8513,4774,2778,2131,...,52698,32231,1154,1982,1460,1870,338,932,834,11897
96,37193,66335,33111,33224,3278,3528,4077,2659,1402,1004,...,31783,21980,23,476,489,487,250,270,398,7410
97,37195,78844,37482,41362,4679,4726,5581,3276,2196,1081,...,36195,23186,1022,2600,1531,1813,330,299,341,5073
98,37197,37198,18636,18562,1837,1901,2629,1443,661,406,...,17013,11857,63,444,132,147,100,281,25,3964


In [ ]:
import glob

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/acs_results/NC/'
files = glob.glob(f'{data_dir}/*.csv')

# which_targets = ['1 wk ahead cum death', '2 wk ahead cum death',
#                  '3 wk ahead cum death', '4 wk ahead cum death']
# which_targets = which_targets[:1]
# types = ['point']

df = None
all_dfs = []
for f in files:
    if not df:
        df = pd.read_csv(f)
        continue
    else:
        this_df = pd.read_csv(f)
        df = df.merge(this_df, on='GEOID')
    
#     df = pd.read_csv(f)
    
    # Choose which time-ahead targets
#     df = df[df.target.isin(which_targets)]
    # Choose which types (only point for now)
#     df = df[df.type.isin(types)]

#     all_dfs.append(df)
    
# tot_df = pd.concat(all_dfs)
# display(tot_df)

In [ ]:
all_dfs[2]

In [ ]:
dff = all_dfs[0].merge(all_dfs[1:], on='GEOID')
dff

### <font color=blue> Get state COVIDhub-ensemble predictions.

In [ ]:
## Get fips
state_to_fips = pd.read_csv('constants/state_fips.csv')
state_to_fips.rename({'FIPS':'State_fips'},axis=1,inplace=True)

## Add fips to forecast table.
df = pd.read_csv(f'{data_dir}covidhub_ensemble_1wkcum_point_20240911-114640.csv')
df.columns = df.columns.str.capitalize()
df.rename({'Location':'State_fips'},axis=1,inplace=True)

df = df.merge(state_to_fips, on='State_fips')

## Remove and rename columns ...
df_states = df.drop(['Target','Forecast_date'], axis=1, errors='ignore')
df_states.rename({'Target_end_date':'Date', 'Value':'COVIDhubEns_state_deaths'}, axis=1, inplace=True)
df_states

### <font color=blue> Get ground truth covid deaths per county.

In [ ]:
df_counties = pd.read_csv(f'{data_dir}nytimes/us-counties.csv')
df_counties.columns = df_counties.columns.str.capitalize()
df_counties.drop(['Cases'],axis=1,inplace=True)
df_counties.dropna(inplace=True)

## Make list of dfs because everything in one df is too big.

forecast_dates = list(df_states.Date.unique())
all_states = list(df_counties.State.unique())

state_dfs = {}

for state in tqdm(all_states, total=len(all_states)):
    df = df_counties[df_counties.State==state]
    df['Fips'] = df['Fips'].astype('int64').astype('str')
    ## Only need dates for counties that we have for states ... 
    df = df[df.Date.isin(forecast_dates)]
    state_dfs[state] = df

<font color=blue> Get pop ratios for each county.

In [ ]:
df_pop = pd.read_csv('cached_data/pop_ratios_per_county.csv')
df_pop['Fips'] = df_pop['Fips'].astype('str')
df_pop

In [ ]:
for state,df in state_dfs.items():
    df_state_pop = df_pop[df_pop.State==state]    
    df = df.merge(df_state_pop, on=['Fips','State'])
    state_dfs[state] = df
# state_dfs['Alabama']

### <font color=blue> Compute naive deaths for each county.

In [ ]:
merged_dfs = {}
for state in tqdm(all_states, total=len(all_states)):
    
    df_state = df_states[df_states.State==state]
    df_counties = state_dfs[state]
    df_merged = df_counties.merge(df_state, on=['Date','State','Postal'])
    df_merged.rename({'Deaths':'True_county_deaths'}, axis=1, inplace=True)
    
    df_merged['Naive_county_deaths'] = df_merged.apply(lambda x: x.Pop_ratio * x.COVIDhubEns_state_deaths, 
                                                       axis=1)
    
    df_merged = df_merged[['State_fips','State','Postal','County','Fips','Date','COVIDhubEns_state_deaths',
                           'Pop','Pop_ratio','True_county_deaths',
                           'Naive_county_deaths']]
    merged_dfs[state] = df_merged

In [ ]:
dff = merged_dfs['North Carolina']
dff = dff[dff.County=='Alexander']
dff = dff[dff.Date=='2021-02-06']
dff

In [ ]:
final_df = pd.concat(merged_dfs.values())
final_df

In [ ]:
data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
final_df.to_csv(f'{data_dir}naive_deaths_all_counties_{d}.csv',index=False)